In [76]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

In [77]:
qqq_prices = pd.read_csv("Data/dataset1.csv", header=0)
qqq_prices = qqq_prices[1:]
qqq_prices.rename(columns={'Date': 'date'}, inplace=True)
qqq_prices = qqq_prices.sort_values(by='date').reset_index(drop=True)
qqq_prices = qqq_prices[qqq_prices['date'].notnull()]
starting_price = 150.0
qqq_prices['qqq_price'] = starting_price * (1 + qqq_prices['Return'].fillna(0)).cumprod()
qqq_prices.head(10)


,date,Return,Realized_Volatility,Volume,VIX,RSI_14,VXN,FedRate,qqq_price
0,2018-02-22,-0.000122,0.283917,37074400,18.719999313354492,45.694807,20.90999984741211,1.6050000190734863,149.981774
1,2018-02-23,0.020449,0.292775,50096900,16.489999771118164,54.847494,18.15999984741211,1.6050000190734863,153.048722
2,2018-02-26,0.013260,0.296289,39266700,15.800000190734863,68.915601,18.1200008392334,1.6080000400543213,155.078209
3,2018-02-27,-0.012383,0.294650,42209900,18.59000015258789,59.842025,20.489999771118164,1.6349999904632568,153.157930
4,2018-02-28,-0.006417,0.294986,42936300,19.850000381469727,61.920940,22.010000228881836,1.6180000305175781,152.175044
5,2018-03-01,-0.016327,0.298914,76809900,22.469999313354492,71.772822,24.739999771118164,1.590000033378601,149.690529
6,2018-03-02,0.009180,0.300518,57445100,19.59000015258789,70.442520,21.530000686645508,1.6050000190734863,151.064755
7,2018-03-05,0.011085,0.301846,38540200,18.729999542236328,69.260689,20.600000381469727,1.625,152.739266
8,2018-03-06,0.004231,0.293070,29480900,18.360000610351562,69.114625,20.600000381469727,1.6430000066757202,153.385444
9,2018-03-07,0.002314,0.252926,34609400,17.760000228881836,65.207063,20.43000030517578,1.649999976158142,153.740420


In [78]:
qqq_options = pd.read_csv("Data/qqq_option_data_2018_2023.csv", header=0)
qqq_options.head(10)

,date,exdate,cp_flag,strike_price,delta,gamma,impl_volatility,best_bid,best_offer,volume,open_interest,optionid,secid
0,2018-01-02,2018-02-02,C,148500.0,0.892498,0.021384,0.186879,10.49,10.70,4.0,40.0,118524041.0,107899.0
1,2018-01-02,2018-02-02,C,149500.0,0.875178,0.024551,0.181047,9.61,9.73,2.0,12.0,118524043.0,107899.0
2,2018-01-02,2018-02-02,C,153000.0,0.791621,0.039376,0.157560,6.48,6.56,3.0,42.0,118524049.0,107899.0
3,2018-01-02,2018-02-02,C,153500.0,0.776244,0.042127,0.153491,6.05,6.11,11.0,98.0,118524050.0,107899.0
4,2018-01-02,2018-02-02,C,154000.0,0.758090,0.044900,0.150400,5.63,5.69,20.0,91.0,118524051.0,107899.0
5,2018-01-02,2018-02-02,C,154500.0,0.738672,0.047811,0.147127,5.22,5.27,11.0,50.0,118524052.0,107899.0
6,2018-01-02,2018-02-02,C,155000.0,0.717501,0.050793,0.143971,4.81,4.87,22.0,72.0,118524053.0,107899.0
7,2018-01-02,2018-02-02,C,156000.0,0.669563,0.056786,0.138024,4.04,4.09,24.0,46.0,118524055.0,107899.0
8,2018-01-02,2018-02-02,C,156500.0,0.643388,0.059961,0.134537,3.66,3.71,66.0,80.0,118524056.0,107899.0
9,2018-01-02,2018-02-02,C,157000.0,0.614649,0.062772,0.131780,3.31,3.35,111.0,593.0,118524057.0,107899.0


In [79]:
zero_coupon_rates = pd.read_csv("Data/zero_coupon_rates_2018_2023.csv")
zero_coupon_rates.head(10)

,date,days,rate
0,2018-01-02,7.0,1.501048
1,2018-01-02,15.0,1.527942
2,2018-01-02,50.0,1.649744
3,2018-01-02,78.0,1.683121
4,2018-01-02,106.0,1.717587
5,2018-01-02,134.0,1.748227
6,2018-01-02,169.0,1.782660
7,2018-01-02,260.0,1.850775
8,2018-01-02,351.0,1.913646
9,2018-01-02,442.0,1.972431


In [80]:
import time

notional_value = 1000000
hedge_results = []

In [83]:
import time

# Simulation parameters
notional_value = 1_000_000
hedge_results = []

for raw_date in sorted(qqq_prices['date'].unique()):
    current_date = pd.to_datetime(raw_date)
    price_row = qqq_prices[qqq_prices['date'] == current_date]
    if price_row.empty or pd.isna(current_date):
        continue

    qqq_price = float(price_row['qqq_price'].values[0])
    shares_held = notional_value / qqq_price
    portfolio_delta = shares_held * 1.0
    portfolio_gamma = 0.0

    option_slice = qqq_options[qqq_options['date'] == current_date]
    option_slice = option_slice[option_slice['exdate'] <= current_date + timedelta(days=30)]
    if option_slice.empty:
        continue

    option_slice = option_slice.dropna(subset=['delta', 'gamma', 'best_bid', 'best_offer'])

    atm_range = (qqq_price * 0.95 * 1000, qqq_price * 1.05 * 1000)  # widen range to ±5%
    atm_options = option_slice[
        (option_slice['strike_price'] >= atm_range[0]) &
        (option_slice['strike_price'] <= atm_range[1])
    ]
    if atm_options.empty:
        continue

    calls = atm_options[atm_options['cp_flag'] == 'C'].sort_values(by='exdate')
    puts = atm_options[atm_options['cp_flag'] == 'P'].sort_values(by='exdate')
    if calls.empty or puts.empty:
        continue

    call_option = calls.iloc[0]
    put_option = puts.iloc[0]

    delta_call = call_option['delta']
    gamma_call = call_option['gamma']
    delta_put = put_option['delta']
    gamma_put = put_option['gamma']

    try:
        A = np.array([[delta_call, delta_put],
                      [gamma_call, gamma_put]])
        b = np.array([-portfolio_delta, -portfolio_gamma])
        hedge_weights = np.linalg.solve(A, b)
        num_calls, num_puts = round(hedge_weights[0]), round(hedge_weights[1])

        call_price = (call_option['best_bid'] + call_option['best_offer']) / 2
        put_price = (put_option['best_bid'] + put_option['best_offer']) / 2
        hedge_cost = (abs(num_calls) * call_price + abs(num_puts) * put_price) * 100

        hedge_results.append({
            'date': current_date,
            'qqq_price': qqq_price,
            'num_calls': num_calls,
            'num_puts': num_puts,
            'call_strike': call_option['strike_price'] / 1000,
            'put_strike': put_option['strike_price'] / 1000,
            'call_expiry': call_option['exdate'],
            'put_expiry': put_option['exdate'],
            'call_price': call_price,
            'put_price': put_price,
            'hedge_cost': hedge_cost
        })
    except np.linalg.LinAlgError:
        continue

hedge_df = pd.DataFrame(hedge_results)


In [84]:
print(hedge_df)

Empty DataFrame
Columns: []
Index: []


In [87]:
# Re-import necessary libraries after reset
import pandas as pd
import numpy as np
from datetime import timedelta

# Reload uploaded files
qqq_options = pd.read_csv("Data/qqq_option_data_2018_2023.csv")
qqq_prices = pd.read_csv("Data/dataset1.csv")

# Clean and standardize columns
qqq_options['date'] = pd.to_datetime(qqq_options['date'], errors='coerce')
qqq_options['exdate'] = pd.to_datetime(qqq_options['exdate'], errors='coerce')
qqq_prices.rename(columns={'Date': 'date'}, inplace=True)
qqq_prices['date'] = pd.to_datetime(qqq_prices['date'], errors='coerce')
qqq_options = qqq_options.dropna(subset=['date', 'exdate'])

# Reconstruct QQQ prices from returns
qqq_prices = qqq_prices.sort_values(by='date').reset_index(drop=True)
qqq_prices = qqq_prices[qqq_prices['date'].notnull()]
starting_price = 150.0
qqq_prices['qqq_price'] = starting_price * (1 + qqq_prices['Return'].fillna(0)).cumprod()

# Simulation parameters
notional_value = 1_000_000
hedge_results = []

# Perform delta-gamma hedging
for raw_date in sorted(qqq_prices['date'].unique()):
    current_date = pd.to_datetime(raw_date)
    price_row = qqq_prices[qqq_prices['date'] == current_date]
    if price_row.empty or pd.isna(current_date):
        continue

    qqq_price = float(price_row['qqq_price'].values[0])
    shares_held = notional_value / qqq_price
    portfolio_delta = shares_held * 1.0
    portfolio_gamma = 0.0

    option_slice = qqq_options[qqq_options['date'] == current_date]
    option_slice = option_slice[option_slice['exdate'] <= current_date + timedelta(days=30)]
    if option_slice.empty:
        continue

    option_slice = option_slice.dropna(subset=['delta', 'gamma', 'best_bid', 'best_offer'])

    atm_range = (qqq_price * 0.95 * 1000, qqq_price * 1.05 * 1000)
    atm_options = option_slice[
        (option_slice['strike_price'] >= atm_range[0]) &
        (option_slice['strike_price'] <= atm_range[1])
    ]
    if atm_options.empty:
        continue

    calls = atm_options[atm_options['cp_flag'] == 'C'].sort_values(by='exdate')
    puts = atm_options[atm_options['cp_flag'] == 'P'].sort_values(by='exdate')
    if calls.empty or puts.empty:
        continue

    call_option = calls.iloc[0]
    put_option = puts.iloc[0]

    delta_call = call_option['delta']
    gamma_call = call_option['gamma']
    delta_put = put_option['delta']
    gamma_put = put_option['gamma']

    try:
        A = np.array([[delta_call, delta_put],
                      [gamma_call, gamma_put]])
        b = np.array([-portfolio_delta, -portfolio_gamma])
        hedge_weights = np.linalg.solve(A, b)
        num_calls, num_puts = round(hedge_weights[0]), round(hedge_weights[1])

        call_price = (call_option['best_bid'] + call_option['best_offer']) / 2
        put_price = (put_option['best_bid'] + put_option['best_offer']) / 2
        hedge_cost = (abs(num_calls) * call_price + abs(num_puts) * put_price) * 100

        hedge_results.append({
            'date': current_date,
            'qqq_price': qqq_price,
            'num_calls': num_calls,
            'num_puts': num_puts,
            'call_strike': call_option['strike_price'] / 1000,
            'put_strike': put_option['strike_price'] / 1000,
            'call_expiry': call_option['exdate'],
            'put_expiry': put_option['exdate'],
            'call_price': call_price,
            'put_price': put_price,
            'hedge_cost': hedge_cost
        })
    except np.linalg.LinAlgError:
        continue

hedge_df = pd.DataFrame(hedge_results)
hedge_df

,date,qqq_price,num_calls,num_puts,call_strike,put_strike,call_expiry,put_expiry,call_price,put_price,hedge_cost
0,2018-02-22,149.981774,-6744,22812,156.0,143.0,2018-03-23,2018-03-23,10.190,0.360,7693368.0
1,2018-02-23,153.048722,-6427,14279,156.0,148.0,2018-03-23,2018-03-23,12.625,0.295,8535318.0
2,2018-02-26,155.078209,-6381,5499,150.0,147.5,2018-03-23,2018-03-23,20.450,0.180,13148127.0
3,2018-02-27,153.157930,-6555,13377,154.0,147.0,2018-03-29,2018-03-29,15.125,0.350,10382632.5
4,2018-02-28,152.175044,-6575,10670,150.0,145.0,2018-03-29,2018-03-29,17.970,0.415,12258080.0
...,...,...,...,...,...,...,...,...,...,...,...
1181,2023-08-24,341.255930,-2976,3226,325.0,325.0,2023-09-22,2023-09-22,38.675,0.975,11824215.0
1182,2023-08-25,343.901152,-2924,3379,330.0,328.0,2023-09-22,2023-09-22,36.000,0.780,10789962.0
1183,2023-08-28,346.489801,-2919,4797,338.0,330.0,2023-09-22,2023-09-22,30.760,0.575,9254671.5
1184,2023-08-30,356.031506,-2811,2908,340.0,339.0,2023-09-29,2023-09-29,38.470,0.655,11004391.0
